In [1]:
import sys, os, json
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
import itertools
from gensim.corpora import Dictionary
from collections import defaultdict
import statsmodels.api as sm
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow, experiment_result
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper

In [2]:
with open('experiments_dataset1.json', 'r') as fout:
    res=json.load(fout)

In [3]:
res.keys()

dict_keys(['lda_experiment', 'gsdmm_experiment', 'hdp_experiment', 'lftm_experiment'])

In [4]:
with open('experiments_dataset2.json', 'r') as fout:
    res2=json.load(fout)

In [5]:
import pandas as pd
def to_df(res):
    params=defaultdict(list)
    for exp in res.keys():
        params['c_we'].append(res[exp]['coherence_metrics']['c_we']['c_we'])
        params['c_we_std'].append(res[exp]['coherence_metrics']['c_we']['c_we_std'])
        #params['num_topics'].append(res[exp]['number_topics'])
        for p in res[exp]['hyperparameters'].keys():
            params[p].append(res[exp]['hyperparameters'][p])
    #if params['num_topics'][0]==None:
    #    del params['num_topics']
    return pd.DataFrame(params)
    #return params


In [6]:
def to_df2(res):
    params=defaultdict(list)
    for exp in res.keys():
        params['u_mass'].append(res[exp]['coherence_metrics']['u_mass']['u_mass'])
        params['u_mass_std'].append(res[exp]['coherence_metrics']['u_mass']['u_mass_std'])
        #params['num_topics'].append(res[exp]['number_topics'])
        for p in res[exp]['hyperparameters'].keys():
            params[p].append(res[exp]['hyperparameters'][p])
    #if params['num_topics'][0]==None:
    #    del params['num_topics']
    return pd.DataFrame(params)

In [7]:
def to_df3(res):
    params=defaultdict(list)
    for exp in res.keys():
        params['excl_we'].append(res[exp]['coherence_metrics']['c_we']['excl_we'])
        params['excl_we_std'].append(res[exp]['coherence_metrics']['c_we']['excl_we_std'])
        #params['num_topics'].append(res[exp]['number_topics'])
        for p in res[exp]['hyperparameters'].keys():
            params[p].append(res[exp]['hyperparameters'][p])
    #if params['num_topics'][0]==None:
    #    del params['num_topics']
    return pd.DataFrame(params)

## LDA: word embedding coherence

#### DATASET 1

In [38]:
lda=to_df(res['lda_experiment'])
lda.iloc[lda.nlargest(5, 'c_we').index]

,c_we,c_we_std,num_topics,decay,passes
23,0.228105,0.063241,13,0.75,5
6,0.227736,0.071636,7,0.90,1
16,0.219584,0.049335,10,0.90,2
17,0.215695,0.064932,10,0.90,5
2,0.213179,0.059625,7,0.50,5


In [43]:
with open('res.txt', 'w') as fout:
    for i in range(10):
        fout.write('\n')
        for t in res2['lda_experiment']['exp_23']['word_topic_pvalues']:
            fout.write(t['words'][i]+' & ')



In [9]:
lda.iloc[lda.nsmallest(5, 'c_we_std').index]

,c_we,c_we_std,num_topics,decay,passes
15,0.170765,0.034695,10,0.9,1
11,0.202779,0.038363,10,0.5,5
18,0.192849,0.039004,13,0.5,1
7,0.188668,0.042965,7,0.9,2
8,0.179143,0.043504,7,0.9,5


In [10]:
import statsmodels.api as sm
y = lda['c_we']
x=lda.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   c_we   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                 -0.069
Method:                 Least Squares   F-statistic:                    0.4409
Date:                Mon, 06 Nov 2023   Prob (F-statistic):              0.726
Time:                        15:00:18   Log-Likelihood:                 78.817
No. Observations:                  27   AIC:                            -149.6
Df Residuals:                      23   BIC:                            -144.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1918      0.017     11.278      0.0

#### DATASET 2


In [14]:
lda2=to_df(res2['lda_experiment'])
lda2.iloc[lda2.nlargest(5, 'c_we').index]

,c_we,c_we_std,num_topics,decay,passes
25,0.245817,0.080681,13,0.9,2
8,0.241227,0.104337,7,0.9,5
16,0.241086,0.091643,10,0.9,2
10,0.239848,0.096336,10,0.5,2
9,0.237105,0.083887,10,0.5,1


In [15]:
with open('res.txt', 'w') as fout:
    for i in range(10):
        fout.write('\n')
        for t in res2['lda_experiment']['exp_25']['word_topic_pvalues']:
            fout.write(t['words'][i]+' & ')

In [9]:
lda2.iloc[lda2.nsmallest(5, 'c_we_std').index]

,c_we,c_we_std,num_topics,decay,passes
12,0.231812,0.079083,10,0.75,1
25,0.245817,0.080681,13,0.90,2
9,0.237105,0.083887,10,0.50,1
7,0.209926,0.091174,7,0.90,2
16,0.241086,0.091643,10,0.90,2


In [16]:
import statsmodels.api as sm
y = lda2['c_we']
x=lda2.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   c_we   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.130
Method:                 Least Squares   F-statistic:                     2.296
Date:                Mon, 06 Nov 2023   Prob (F-statistic):              0.105
Time:                        17:45:17   Log-Likelihood:                 69.101
No. Observations:                  27   AIC:                            -130.2
Df Residuals:                      23   BIC:                            -125.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2505      0.024     10.277      0.0

In [12]:
y = lda2['c_we_std']
x=lda2.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               c_we_std   R-squared:                       0.330
Model:                            OLS   Adj. R-squared:                  0.243
Method:                 Least Squares   F-statistic:                     3.781
Date:                Mon, 06 Nov 2023   Prob (F-statistic):             0.0243
Time:                        17:41:01   Log-Likelihood:                 79.610
No. Observations:                  27   AIC:                            -151.2
Df Residuals:                      23   BIC:                            -146.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0674      0.017      4.079      0.0

## LDA: UMASS coherence

#### dataset 1


In [50]:
lda=to_df2(res['lda_experiment'])
lda.iloc[lda.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,num_topics,decay,passes
8,-2.689002,0.921021,7,0.90,5
12,-2.842383,0.833301,10,0.75,1
20,-2.882799,0.758300,13,0.50,5
13,-2.893313,0.436060,10,0.75,2
4,-2.910037,0.498769,7,0.75,2


In [51]:
with open('res.txt', 'w') as fout:
    for i in range(10):
        fout.write('\n')
        for t in res2['lda_experiment']['exp_8']['word_topic_pvalues']:
            fout.write(t['words'][i]+' & ')

In [46]:
lda.iloc[lda.nsmallest(5, 'u_mass_std').index]

,u_mass,u_mass_std,num_topics,decay,passes
15,0.214349,0.018445,10,0.90,1
5,0.236633,0.022921,7,0.75,5
8,0.198883,0.024456,7,0.90,5
1,0.216574,0.024633,7,0.50,2
18,0.221758,0.024796,13,0.50,1


In [14]:
y = lda['u_mass']
x=lda.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 u_mass   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                    0.8036
Date:                Mon, 06 Nov 2023   Prob (F-statistic):              0.505
Time:                        16:01:21   Log-Likelihood:                 4.7188
No. Observations:                  27   AIC:                            -1.438
Df Residuals:                      23   BIC:                             3.746
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.4413      0.265    -13.008      0.0

In [15]:
y = lda['u_mass_std']
x=lda.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             u_mass_std   R-squared:                       0.211
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     2.046
Date:                Mon, 06 Nov 2023   Prob (F-statistic):              0.135
Time:                        16:01:24   Log-Likelihood:                -3.4295
No. Observations:                  27   AIC:                             14.86
Df Residuals:                      23   BIC:                             20.04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9240      0.358      2.583      0.0

### LDA exclusivity metric

In [55]:
lda=to_df3(res['lda_experiment'])
lda.iloc[lda.nsmallest(5, 'excl_we').index]

,excl_we,excl_we_std,num_topics,decay,passes
8,0.198883,0.024456,7,0.90,5
11,0.208448,0.027347,10,0.50,5
26,0.209971,0.027718,13,0.90,5
10,0.210594,0.037027,10,0.50,2
12,0.210659,0.029742,10,0.75,1


In [ ]:
with open('res.txt', 'w') as fout:
    for i in range(10):
        fout.write('\n')
        for t in res2['lda_experiment']['exp_8']['word_topic_pvalues']:
            fout.write(t['words'][i]+' & ')

In [37]:
y = lda['excl_we']
x=lda.drop(columns=['excl_we', 'excl_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                excl_we   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                 -0.035
Method:                 Least Squares   F-statistic:                    0.7084
Date:                Mon, 06 Nov 2023   Prob (F-statistic):              0.557
Time:                        16:32:17   Log-Likelihood:                 81.294
No. Observations:                  27   AIC:                            -154.6
Df Residuals:                      23   BIC:                            -149.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2094      0.016     13.494      0.0

#### dataset 2

In [147]:
lda2=to_df2(res2['lda_experiment'])
lda2.iloc[lda2.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,num_topics,decay,passes
25,-0.461509,0.377412,10,0.90,2
4,-0.554065,0.100125,6,0.75,2
3,-0.570491,0.343090,6,0.75,1
17,-0.587406,0.492029,8,0.90,5
0,-0.594014,0.434840,6,0.50,1


In [156]:
res2['lda_experiment']['exp_4']['word_topic_pvalues'][3]['words']

['claim',
 'covid',
 'fact',
 'say',
 'video',
 'check',
 'use',
 'share',
 'news',
 'story']

In [148]:
lda2.iloc[lda2.nsmallest(5, 'u_mass_std').index]

,u_mass,u_mass_std,num_topics,decay,passes
4,-0.554065,0.100125,6,0.75,2
15,-0.653324,0.316765,8,0.90,1
3,-0.570491,0.343090,6,0.75,1
46,-0.713196,0.352016,16,0.50,2
25,-0.461509,0.377412,10,0.90,2


In [149]:
y = lda2['u_mass']
x=lda2.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 u_mass   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.274
Date:                Fri, 03 Nov 2023   Prob (F-statistic):              0.292
Time:                        09:46:37   Log-Likelihood:                -12.349
No. Observations:                  63   AIC:                             32.70
Df Residuals:                      59   BIC:                             41.27
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8466      0.215     -3.947      0.0

In [112]:
y = lda2['u_mass_std']
x=lda2.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             u_mass_std   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     1.553
Date:                Thu, 02 Nov 2023   Prob (F-statistic):              0.210
Time:                        15:11:34   Log-Likelihood:                -54.773
No. Observations:                  63   AIC:                             117.5
Df Residuals:                      59   BIC:                             126.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7614      0.421      1.810      0.0

## GSDMM: word embedding coherence

#### dataset 1

In [139]:
gsdmm=to_df(res['gsdmm_experiment'])
gsdmm.iloc[gsdmm.nlargest(5, 'c_we').index]

,c_we,c_we_std,K,alpha,beta,n_iters
1,0.365417,0.0,6,0.1,0.1,5
34,0.365417,0.0,12,0.2,0.2,5
2,0.352392,0.0,6,0.1,0.1,15
13,0.332914,0.0,9,0.1,0.1,5
61,0.322158,0.0,21,0.1,0.1,5


In [115]:
gsdmm.iloc[gsdmm.nsmallest(5, 'c_we_std').index]

,c_we,c_we_std,K,alpha,beta,n_iters
0,0.184020,0.0,6,0.1,0.1,1
1,0.365417,0.0,6,0.1,0.1,5
2,0.352392,0.0,6,0.1,0.1,15
3,0.183047,0.0,6,0.1,0.2,1
4,0.240072,0.0,6,0.1,0.2,5


In [117]:
y = gsdmm['c_we']
x=gsdmm.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   c_we   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     2.273
Date:                Thu, 02 Nov 2023   Prob (F-statistic):             0.0687
Time:                        15:14:52   Log-Likelihood:                 118.37
No. Observations:                  84   AIC:                            -226.7
Df Residuals:                      79   BIC:                            -214.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3047      0.034      8.869      0.0

#### dataset 2

In [136]:
gsdmm=to_df2(res2['gsdmm_experiment'])
gsdmm.iloc[gsdmm.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,K,alpha,beta,n_iters
4,-0.509983,0.054108,6,0.1,0.2,5
43,-0.636831,0.184751,15,0.2,0.1,5
26,-0.683594,0.249467,12,0.1,0.1,15
50,-0.687103,0.254352,18,0.1,0.1,15
65,-0.733169,0.318711,21,0.1,0.2,15


In [137]:
gsdmm.iloc[gsdmm.nsmallest(5, 'u_mass_std').index]

,u_mass,u_mass_std,K,alpha,beta,n_iters
4,-0.509983,0.054108,6,0.1,0.2,5
43,-0.636831,0.184751,15,0.2,0.1,5
26,-0.683594,0.249467,12,0.1,0.1,15
50,-0.687103,0.254352,18,0.1,0.1,15
65,-0.733169,0.318711,21,0.1,0.2,15


In [138]:
y = gsdmm['u_mass']
x=gsdmm.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 u_mass   R-squared:                       0.334
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     9.916
Date:                Fri, 03 Nov 2023   Prob (F-statistic):           1.49e-06
Time:                        08:27:35   Log-Likelihood:                -18.533
No. Observations:                  84   AIC:                             47.07
Df Residuals:                      79   BIC:                             59.22
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0336      0.175     -5.897      0.0

## GSDMM: UMASS coherence

#### dataset 1

In [145]:
gsdmm=to_df2(res['gsdmm_experiment'])
gsdmm.iloc[gsdmm.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,K,alpha,beta,n_iters
2,-2.448810,0.978883,6,0.1,0.1,15
1,-2.456073,0.978553,6,0.1,0.1,5
10,-2.472700,1.012691,6,0.2,0.2,5
8,-2.474787,0.989685,6,0.2,0.1,15
0,-2.549945,0.977607,6,0.1,0.1,1


In [146]:
y = gsdmm['u_mass_std']
x=gsdmm.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             u_mass_std   R-squared:                       0.528
Model:                            OLS   Adj. R-squared:                  0.504
Method:                 Least Squares   F-statistic:                     22.12
Date:                Fri, 03 Nov 2023   Prob (F-statistic):           2.81e-12
Time:                        09:09:27   Log-Likelihood:                -90.780
No. Observations:                  84   AIC:                             191.6
Df Residuals:                      79   BIC:                             203.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5306      0.414     -1.281      0.2

#### dataset 2

In [143]:
gsdmm=to_df2(res2['gsdmm_experiment'])
gsdmm.iloc[gsdmm.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,K,alpha,beta,n_iters
4,-0.509983,0.054108,6,0.1,0.2,5
43,-0.636831,0.184751,15,0.2,0.1,5
26,-0.683594,0.249467,12,0.1,0.1,15
50,-0.687103,0.254352,18,0.1,0.1,15
65,-0.733169,0.318711,21,0.1,0.2,15


In [144]:
y = gsdmm['u_mass_std']
x=gsdmm.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             u_mass_std   R-squared:                       0.121
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     2.728
Date:                Fri, 03 Nov 2023   Prob (F-statistic):             0.0349
Time:                        08:57:34   Log-Likelihood:                -8.3064
No. Observations:                  84   AIC:                             26.61
Df Residuals:                      79   BIC:                             38.77
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5481      0.155      3.532      0.0

### LDA: cluster title generation

In [97]:
from transformers import pipeline
from bertopic.representation import TextGeneration

def prompt(topicwords):
    prompt= f"I have a topic described by the following keywords: [{', '.join(map(str, topicwords))}]. Based on the previous keywords, what is this topic about?"
    return prompt

# Create your representation model
generator = pipeline('text2text-generation', model='google/flan-t5-base')
representation_model = TextGeneration(generator)